# Import modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# ex: from src.create_model_pickle import *


# Import data

In [3]:
# Use our json to dataframe conversion class
from src.pipeline import pipeline_json
pj = pipeline_json('../data/data.json')

In [48]:
df = pj.convert_to_df(scaling=True, filtered=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 15 columns):
fraud                  14337 non-null int64
payout_type            14337 non-null int64
fb_published           14337 non-null int64
org_facebook           14337 non-null float64
has_analytics          14337 non-null float64
has_header             14337 non-null float64
org_twitter            14337 non-null float64
account_life           14337 non-null float64
event_life             14337 non-null float64
eu_currency            14337 non-null int64
payout_count           14337 non-null float64
total_payout           14337 non-null float64
ticket_sales_amount    14337 non-null float64
ticket_sales_count     14337 non-null float64
wc_description         14337 non-null float64
dtypes: float64(11), int64(4)
memory usage: 1.8 MB


In [50]:
df.describe()

,fraud,payout_type,fb_published,org_facebook,has_analytics,has_header,org_twitter,account_life,event_life,eu_currency,payout_count,total_payout,ticket_sales_amount,ticket_sales_count,wc_description
count,14337.000000,14337.000000,14337.000000,1.433700e+04,1.433700e+04,14337.000000,1.433700e+04,1.433700e+04,1.433700e+04,14337.000000,1.433700e+04,1.433700e+04,1.433700e+04,1.433700e+04,1.433700e+04
mean,0.090186,0.965613,0.127433,-8.005345e-16,9.346642e-16,-0.251447,7.391885e-16,2.965706e-16,3.772675e-15,0.173258,1.126291e-16,7.708140e-17,-5.693600e-17,2.669274e-17,1.351286e-17
std,0.286458,0.182226,0.333468,1.000035e+00,1.000035e+00,0.663256,1.000035e+00,1.000035e+00,1.000035e+00,0.378483,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00,1.000035e+00
min,0.000000,0.000000,0.000000,-7.605960e-01,-2.806796e-01,-1.000000,-8.796040e-01,-8.932491e-01,-6.751056e-01,0.000000,-3.202421e-01,-1.744858e-01,-2.021541e-01,-1.927533e-01,-8.124857e-01
25%,0.000000,1.000000,0.000000,-6.754944e-01,-2.806796e-01,-1.000000,-7.131743e-01,-8.645752e-01,-5.921202e-02,0.000000,-3.118750e-01,-1.732202e-01,-1.963488e-01,-1.841894e-01,-5.579209e-01
50%,0.000000,1.000000,0.000000,-6.754944e-01,-2.806796e-01,0.000000,-7.131743e-01,-3.173805e-01,-5.921202e-02,0.000000,-2.867738e-01,-1.639917e-01,-1.754496e-01,-1.527884e-01,-2.426522e-01
75%,0.000000,1.000000,0.000000,5.159282e-01,-2.806796e-01,0.000000,1.117553e+00,5.452277e-01,-5.921202e-02,0.000000,-1.696349e-01,-1.112408e-01,-9.718430e-02,-6.429459e-02,2.442846e-01
max,1.000000,1.000000,1.000000,7.068752e+00,3.562781e+00,1.000000,4.945438e+00,8.170110e+00,1.634511e+01,1.000000,8.247634e+00,2.967627e+01,6.284213e+01,5.695712e+01,4.168156e+01


In [33]:
df['event_life'] = df['event_life'].map(lambda x: 0 if np.isnan(x) else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 15 columns):
fraud                  14337 non-null bool
payout_type            14337 non-null int64
fb_published           14337 non-null bool
org_facebook           14337 non-null float64
has_analytics          14337 non-null float64
has_header             14337 non-null float64
org_twitter            14337 non-null float64
account_life           14337 non-null float64
event_life             14337 non-null float64
eu_currency            14337 non-null int64
payout_count           14337 non-null float64
total_payout           14337 non-null float64
ticket_sales_amount    14337 non-null float64
ticket_sales_count     14337 non-null float64
wc_description         14337 non-null float64
dtypes: bool(2), float64(11), int64(2)
memory usage: 2.2 MB


date_cols = ['approx_payout_date', 'event_created', 'event_created', 'event_end', \
                     'event_published', 'event_start', 'user_created']

for col in date_cols:
    print df[col][:5] - df['user_created'][:5]

In [ ]:
df['has_header'][:10]
    

a = df['event_published'][:5] - df['user_created'][:5]
a.var

i=12
total_payout = 0
payout_count = len(df['previous_payouts'][i])
payees = set()
for d in df['previous_payouts'][i]:
    total_payout += d['amount']
    payees.add(d['name'])
print total_payout, payout_count, payees

In [ ]:
a = set()

In [ ]:
df['ticket_types'][0][0].keys()

In [ ]:
Counter(df['fraud'])

In [ ]:
col = 'ticket_sales_amount'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

In [ ]:
col = 'ticket_sales_count'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

In [ ]:

col = 'body_length'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

In [ ]:
df['short_description'][:10]

In [ ]:
df['body_length'][:10]

In [ ]:
Counter(df['currency'])

In [ ]:
# table = pivot_table(df, values='D', index=['A', 'B'],
# ...                     columns=['C'], aggfunc=np.sum)
table = pd.pivot_table(df, columns=['currency'], values=['fraud'], aggfunc=np.mean)
print table

In [ ]:
Counter(df['currency'])

In [ ]:
col = 'ticket_sales_events'
print col
print "fraud :", (df[df['fraud'] == True]['event_created'] - df[df['fraud'] == True]['user_created']).mean()
print "not fraud :", (df[df['fraud'] == False]['event_created'] - df[df['fraud'] == False]['user_created']).mean()


In [ ]:
col = 'has_analytics'
print col
print "fraud :", (df[df['fraud'] == True][col]).mean()
print "not fraud :", (df[df['fraud'] == False][col]).mean()


In [ ]:
import bs4

In [14]:
df['event_life'].map(lambda x: 0 if np.isnan(x))

SyntaxError: invalid syntax (<ipython-input-14-23fe34dca953>, line 1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 16 columns):
fraud                  14337 non-null bool
short_description      14337 non-null bool
payout_type            14337 non-null int64
fb_published           14337 non-null bool
org_facebook           14337 non-null float64
has_analytics          14337 non-null float64
has_header             14337 non-null float64
org_twitter            14337 non-null float64
account_life           14337 non-null float64
event_life             14238 non-null float64
eu_currency            14337 non-null int64
payout_count           14337 non-null float64
total_payout           14337 non-null float64
ticket_sales_amount    14337 non-null float64
ticket_sales_count     14337 non-null float64
wc_description         14337 non-null float64
dtypes: bool(3), float64(11), int64(2)
memory usage: 1.6 MB


In [19]:
df['event_life'][:10]

0   -5.0
1   -1.0
2   -1.0
3   -1.0
4   -1.0
5   -1.0
6   -1.0
7   -1.0
8   -6.0
9   -1.0
Name: event_life, dtype: float64

# Setup PostgreSQL table

In [49]:
#Write sample data to json file
with open('temp.json', 'w') as f:
    f.write(df[:10].to_json(orient='index'))